In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import re
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [4]:
train = pd.read_csv('train_2kmZucJ.csv')
test = pd.read_csv('test_oJQbWVk.csv')
submission = pd.read_csv('sample_submission_LnhVWA4.csv')

In [5]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [6]:
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [7]:
submission.head()

,id,label
0,7921,0
1,7922,0
2,7923,0
3,7924,0
4,7925,0


In [8]:
train.label.value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [9]:
train['tweet'][786]

'iPhone 8 #apple #iphone8 #iphone #iphone7 #iphone7plus #applewatch #applewatch2 … https://www.instagram.com/p/BOLdWr_jyeU/'

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\deepusuresh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\deepusuresh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\deepusuresh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [11]:
stop_words = set(stopwords.words('english'))
stop = [x.lower() for x in stop_words]
lemma = WordNetLemmatizer()

shortcuts = {'u': 'you', 'y': 'why', 'r': 'are', 'doin': 'doing', 'hw': 'how', 'k': 'okay', 'm': 'am', 'b4': 'before',
            'idc': "i do not care", 'ty': 'thankyou', 'wlcm': 'welcome', 'bc': 'because', '<3': 'love', 'xoxo': 'love',
            'ttyl': 'talk to you later', 'gr8': 'great', 'bday': 'birthday', 'awsm': 'awesome', 'gud': 'good', 'h8': 'hate',
            'lv': 'love', 'dm': 'direct message', 'rt': 'retweet', 'wtf': 'hate', 'idgaf': 'hate',
             'irl': 'in real life', 'yolo': 'you only live once'}

def clean(text):
    text = text.lower()
    # keep alphanumeric characters only
    text = re.sub('\W+', ' ', text).strip()
    text = text.replace('user', '')
    # tokenize
    text_token = word_tokenize(text)
    # replace shortcuts using dict
    full_words = []
    for token in text_token:
        if token in shortcuts.keys():
            token = shortcuts[token]
        full_words.append(token)
#     text = " ".join(full_words)
#     text_token = word_tokenize(text)
    # stopwords removal
#     words = [word for word in full_words if word not in stop]
    words_alpha = [re.sub(r'\d+', '', word) for word in full_words]
    words_big = [word for word in words_alpha if len(word)>2]
    stemmed_words = [lemma.lemmatize(word) for word in words_big]
    # join list elements to string
    clean_text = " ".join(stemmed_words)
    clean_text = clean_text.replace('   ', ' ')
    clean_text = clean_text.replace('  ', ' ')
    return clean_text

In [12]:
hypocrite = []
for i in range(len(train['tweet'])):
    if 'hypocrite' in train['tweet'][i]:
        if train['label'][i] == 1:
            hypocrite.append('racist')
        else:
            hypocrite.append('good')
    else:
        hypocrite.append('good')
df = pd.DataFrame(columns=['hypocrite'], data=hypocrite)
print(df['hypocrite'].value_counts())

train['hypocrite'] = hypocrite

good    7920
Name: hypocrite, dtype: int64


In [13]:
train['combined'] = train['tweet'].apply(str) + ' ' + train['hypocrite'].apply(str)

In [14]:
X_train = train.combined
y = train.label
X_test = test.tweet

In [15]:
clean_Xtrain = X_train.apply(lambda x: clean(x))

In [16]:
clean_Xtrain[1531]

'photo helenabonhamcarter actress same necklace apple idol perfect helenaist perfelena http tmblr zfwcodefjf good'

In [17]:
clean_Xtest = X_test.apply(lambda x: clean(x))

In [18]:
print(len(clean_Xtrain))
print(len(clean_Xtest))
print(len(y))

7920
1953
7920


In [19]:
vectorizer = CountVectorizer(max_df=0.5)
# vectorizer = TfidfVectorizer(ngram_range=(1,3), max_df=0.5)

X = vectorizer.fit_transform(clean_Xtrain)
X_test = vectorizer.transform(clean_Xtest)

In [20]:
print(X.shape)
print(X_test.shape)

(7920, 20733)
(1953, 20733)


In [21]:
model = LinearSVC(penalty='l2', C=0.5, dual=False, random_state=0, max_iter=1000)
print(model)

LinearSVC(C=0.5, dual=False, random_state=0)


In [22]:
# split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=0)

# calculate f1 score
model.fit(X_train,y_train)
y_pred = model.predict(X_val)
print('Accuracy:', accuracy_score(y_pred, y_val))
print("F1 Score: ", f1_score(y_pred, y_val))

Accuracy: 0.8724747474747475
F1 Score:  0.7475000000000002


In [23]:
df = pd.DataFrame()
df['y_pred'] = y_pred
df['y_pred'].value_counts()

0    1190
1     394
Name: y_pred, dtype: int64

In [24]:
# train model with full data and predict for new samples
model.fit(X, y)
y_pred = model.predict(X_test)

In [25]:
df = pd.DataFrame()
df['y_pred'] = y_pred
df['y_pred'].value_counts()

0    1429
1     524
Name: y_pred, dtype: int64

In [26]:
# save it to submission csv
submission['label'] = y_pred
submission.to_csv('submission.csv', index=False)